# Hourly check-in histogram heights
For the main histogram

In [7]:
import pandas as pd
import sqlite3
from datetime import datetime, date, time, timedelta

# Set up connection to the SQLite database
connection = sqlite3.connect('foursquare_checkins.db')

# Print all tables
all_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", connection)

# Print information about 'docs' table
docs = pd.read_sql("SELECT * FROM %s" % ("checkins"), connection)

months = range(1,13)
days = range(0,7)
hours = range(0,24)

# Initialize with zeros
hist = {}
for month in months:
    hist[month] = {}
    for day in days:
        hist[month][day] = {}
        for hour in hours:
            hist[month][day][hour] = 0

for i in range(len(docs)):
    checkin_datetime_raw = docs.time[i]
    time_offset = docs.time_offset[i]

    # Remove the decimal seconds
    checkin_datetime_raw = checkin_datetime_raw.split(".")[0].split(" ")

    # Interpret datetimes
    checkin_date = checkin_datetime_raw[0].split("-")
    checkin_time = checkin_datetime_raw[1].split(":")
    
    # Convert to actual datetime
    checkin_datetime = datetime(int(checkin_date[0]), int(checkin_date[1]), 
                                int(checkin_date[2]), int(checkin_time[0]), int(checkin_time[1])) \
                        + timedelta(minutes = time_offset)

    w = checkin_datetime.weekday()
    m = checkin_datetime.month
    h = checkin_datetime.hour
    
    hist[m][w][h] += 1

# Clean up connection to the database
connection.close()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('fivethirtyeight')

x = range(0,24,1)
y = [0]*24

# Combine all the days into one
for day in range(0,7):
    for hour in hist[10][day]:
        y[hour] = y[hour] + hist[10][day][hour]['count']
#         y.append(hist[10][day][hour]['count'])

fig, ax = plt.subplots()
ax = plt.subplot(111)
ax.bar(x, y, width=1)
plt.show()
# plt.close('all')


In [8]:
import csv
cols = ["Month", "Day", "Hour", "Count"]
f = open('data/checkins_hourly.csv', 'wb')
csv_write_object = csv.DictWriter(f , fieldnames=cols)
csv_write_object.writeheader()

for month, weekday_dict in hist.items():
    for weekday, hour_dict in weekday_dict.items():
        for hour, count in hour_dict.items():
            csv_write_object.writerow({"Month": month, "Day": weekday, "Hour": hour, "Count": count})
f.close()

# Half Hourly Checkins

In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime, date, time, timedelta

# Set up connection to the SQLite database
connection = sqlite3.connect('foursquare_checkins.db')

# Print all tables
all_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", connection)

# Print information about 'docs' table
docs = pd.read_sql("SELECT * FROM %s" % ("checkins"), connection)

hist1 = {}
for i in range(len(docs)):
    checkin_datetime_raw = docs.time[i]
    time_offset = docs.time_offset[i]

    # Remove the decimal seconds
    checkin_datetime_raw = checkin_datetime_raw.split(".")[0].split(" ")

    # Interpret datetimes
    checkin_date = checkin_datetime_raw[0].split("-")
    checkin_time = checkin_datetime_raw[1].split(":")
    
    # Convert to actual datetime
    checkin_datetime = datetime(int(checkin_date[0]), int(checkin_date[1]), 
                                int(checkin_date[2]), int(checkin_time[0]), int(checkin_time[1])) \
                        + timedelta(minutes = time_offset)

    w = checkin_datetime.weekday()
    m = checkin_datetime.month
    h = checkin_datetime.hour
    minute = checkin_datetime.minute
    if m not in hist1:
        hist1[m] = {}
    if w not in hist1[m]:
        hist1[m][w] = {}
    if h not in hist1[m][w]:
        hist1[m][w][h] = {}
    if minute < 30 and 0 not in hist1[m][w][h]:
            hist1[m][w][h][0] = {}
            hist1[m][w][h][0] = 0
    if minute >= 30 and 1 not in hist1[m][w][h]:
            hist1[m][w][h][1] = {}
            hist1[m][w][h][1] = 0    
    if minute < 30:
        hist1[m][w][h][0] += 1
    else:
        hist1[m][w][h][1] += 1

# Clean up connection to the database
connection.close()

In [ ]:
import csv
cols = ["Month", "Day", "Hour", "Minute", "Count"]
f = open('data/checkins_half-hourly.csv', 'w')
csv_write_object = csv.DictWriter(f, fieldnames=cols)
csv_write_object.writeheader()

for month, monthitem in hist1.items():
    for weekday, weekdayitem in monthitem.items():
        for hour, houritem in weekdayitem.items():
            for minute, count in houritem.items():
                csv_write_object.writerow({"Month": month, "Day": weekday, "Hour": hour, "Minute": minute, "Count": count})
f.close()

# Histogram by Category
For small multiples

In [9]:
import pandas as pd
import sqlite3
from datetime import datetime, date, time, timedelta

# Set up connection to the SQLite database
connection = sqlite3.connect('foursquare_checkins.db')

# Print all tables
all_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", connection)

# Print information about 'docs' table
docs = pd.read_sql("SELECT * FROM %s" % ("checkins"), connection)

hours = range(0,24)
hist_c = {}

for i in range(len(docs)):
    checkin_datetime_raw = docs.time[i]
    time_offset = docs.time_offset[i]
    category = docs.cat_name[i]

    # Remove the decimal seconds
    checkin_datetime_raw = checkin_datetime_raw.split(".")[0].split(" ")

    # Interpret datetimes
    checkin_date = checkin_datetime_raw[0].split("-")
    checkin_time = checkin_datetime_raw[1].split(":")
    
    # Convert to actual datetime
    checkin_datetime = datetime(int(checkin_date[0]), int(checkin_date[1]), 
                                int(checkin_date[2]), int(checkin_time[0]), int(checkin_time[1])) \
                        + timedelta(minutes = time_offset)

#     w = checkin_datetime.weekday()
#     m = checkin_datetime.month
    h = checkin_datetime.hour
    
    if category not in hist_c:
        hist_c[category] = {}
        for hour in hours:
            hist_c[category][hour] = 0

    hist_c[category][h] += 1

# Clean up connection to the database
connection.close()

In [11]:
import csv
cols = ["Category", "Hour", "Count"]
f = open('data/checkins_category.csv', 'wb')
csv_write_object = csv.DictWriter(f, fieldnames=cols)
csv_write_object.writeheader()

for category, category_dict in hist_c.items():
    for hour, count in category_dict.items():
        csv_write_object.writerow({"Category": category, "Hour": hour, "Count": count})
f.close()

# Histogram by Day

In [16]:
import pandas as pd
import sqlite3
from datetime import datetime, date, time, timedelta

# Set up connection to the SQLite database
connection = sqlite3.connect('foursquare_checkins.db')

# Print all tables
all_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", connection)

# Print information about 'docs' table
docs = pd.read_sql("SELECT * FROM %s" % ("checkins"), connection)

days = range(0,7)
hours = range(0,24)

# Initialize with zeros
hist_d = {}
for day in days:
    hist_d[day] = {}
    for hour in hours:
        hist_d[day][hour] = 0

for i in range(len(docs)):
    checkin_datetime_raw = docs.time[i]
    time_offset = docs.time_offset[i]

    # Remove the decimal seconds
    checkin_datetime_raw = checkin_datetime_raw.split(".")[0].split(" ")

    # Interpret datetimes
    checkin_date = checkin_datetime_raw[0].split("-")
    checkin_time = checkin_datetime_raw[1].split(":")
    
    # Convert to actual datetime
    checkin_datetime = datetime(int(checkin_date[0]), int(checkin_date[1]), 
                                int(checkin_date[2]), int(checkin_time[0]), int(checkin_time[1])) \
                        + timedelta(minutes = time_offset)

    w = checkin_datetime.weekday()
    h = checkin_datetime.hour
    
    hist_d[w][h] += 1

# Clean up connection to the database
connection.close()

In [17]:
import csv
cols = ["Day", "Hour", "Count"]
f = open('data/checkins_day_halfhour.csv', 'wb')
csv_write_object = csv.DictWriter(f, fieldnames=cols)
csv_write_object.writeheader()

for day, day_dict in hist_d.items():
    for hour, count in day_dict.items():
        csv_write_object.writerow({"Day": day, "Hour": hour, "Count": count})
f.close()

# Histogram by Month

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, date, time, timedelta

# Set up connection to the SQLite database
connection = sqlite3.connect('foursquare_checkins.db')

# Print all tables
all_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", connection)

# Print information about 'docs' table
docs = pd.read_sql("SELECT * FROM %s" % ("checkins"), connection)

months = range(1,13)
hours = range(0,24)

# Initialize with zeros
hist_m = {}
for month in months:
    hist_m[month] = {}
    for hour in hours:
        hist_m[month][hour] = 0

for i in range(len(docs)):
    checkin_datetime_raw = docs.time[i]
    time_offset = docs.time_offset[i]

    # Remove the decimal seconds
    checkin_datetime_raw = checkin_datetime_raw.split(".")[0].split(" ")

    # Interpret datetimes
    checkin_date = checkin_datetime_raw[0].split("-")
    checkin_time = checkin_datetime_raw[1].split(":")
    
    # Convert to actual datetime
    checkin_datetime = datetime(int(checkin_date[0]), int(checkin_date[1]), 
                                int(checkin_date[2]), int(checkin_time[0]), int(checkin_time[1])) \
                        + timedelta(minutes = time_offset)

    m = checkin_datetime.month
    h = checkin_datetime.hour
    
    hist_m[m][h] += 1

# Clean up connection to the database
connection.close()

In [3]:
import csv
cols = ["Month", "Hour", "Count"]
f = open('data/checkins_month.csv', 'wb')
csv_write_object = csv.DictWriter(f, fieldnames=cols)
csv_write_object.writeheader()

for month, month_dict in hist_m.items():
    for hour, count in month_dict.items():
        csv_write_object.writerow({"Month": month, "Hour": hour, "Count": count})
f.close()

# Collapse all days into one day
For CartoDB Torque visualization

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, date, time, timedelta
import csv

# Set up connection to the SQLite database
connection = sqlite3.connect('foursquare_checkins.db')

# Print all tables
all_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", connection)

# Print information about 'docs' table
docs = pd.read_sql("SELECT * FROM %s" % ("checkins"), connection)
# print(list(docs.columns.values))
hours = range(0,24)

cols = ["Time", "latitude", "longitude"]
f = open('HourlyCollapse_.csv', 'wb')
csv_write_object = csv.DictWriter(f, fieldnames=cols)
csv_write_object.writeheader()

for i in range(len(docs)):
    checkin_datetime_raw = docs.time[i]
    time_offset = docs.time_offset[i]

    # Remove the decimal seconds
    checkin_datetime_raw = checkin_datetime_raw.split(".")[0].split(" ")

    # Interpret datetimes
    checkin_date = checkin_datetime_raw[0].split("-")
    checkin_time = checkin_datetime_raw[1].split(":")
    
    # Convert to actual datetime
    checkin_datetime = datetime(int(checkin_date[0]), int(checkin_date[1]), 
                                int(checkin_date[2]), int(checkin_time[0]), int(checkin_time[1])) \
                        + timedelta(minutes = -time_offset)
    checkin_datetime = checkin_datetime.replace(year=2012, month=1, day=1)

    dt = checkin_datetime.isoformat()
    csv_write_object.writerow({"Time": dt, "latitude": round(docs.latitude[i],6), "longitude": round(docs.longitude[i],6)})

# Clean up connection to the database
connection.close()
f.close()